<a href="https://colab.research.google.com/github/CristopherCano/Proyecto_Python_Procesamiento_de_datos/blob/main/Proyecto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto de Procesamiento de Datos con Python - BEDU - Módulo 3
## Viajes en Taxis y Ubers en la Zona Metropolitana del Valle de México 2016 - 2017
* Eduardo Alán Hernández Villasana
* Crsitopher Arvizu Cano
* David Gil Peralta
* José Antonio Aguilar Télez

## Planteamiento del Problema
En los últimos años, desde la llegada de aplicaciones para el transporte privado como Uber o Didi, los gremios taxistas de la Zona Metropolitana del Valle de México han protestado en contra de estas aplicaciones, ellos argumentan una competencia desleal y exigen regulaciones hacia estos transportes. Esto no es de extrañarse ya que, con el aumento de la popularidad de estas aplicaciones de transporte, los taxistas están perdiendo clientes y no pueden o quieren modernizarse para poder competir con los automóviles de Uber o Didi que son más modernos. 

A raíz de este enfrentamiento, han surgido noticias comparando estos dos modelos de transporte privado, sin embargo, muchas de estas noticias están sesgadas por intereses políticos y económicos. Es por esta razón que decidimos analizar datos recopilados tanto en Taxis y Ubers para hacer una comparación entre los dos modelos y aportar nuestros hallazgos estadísticos a la discusión de este problema.

## Preguntas Clave
Durante el desarrollo de este proyecto, se espera responder las siguientes preguntas.

* ¿Qué tipo de transporte pasa menos tiempo en el tráfico?
* ¿Cuál es la velocidad promedio de cada tipo de transporte?
* ¿Qué día de la semana hay mas demanda para cada tipo de transporte?
* ¿En qué tipo de transporte recibe más dinero en promedio por viaje?
* ¿Desde qué demsarcación territorial o municipio cada tipo de transporte tiene mayor demanda?
* ¿Cuál es el destino mas solicitado para cada tipo de transporte?

In [1]:
# Librerías a usar
import pandas as pd
import numpy as np

### Extracción de datos
Se descargará la base de datos desde GitHub.

In [2]:
url = "https://raw.githubusercontent.com/CristopherCano/Proyecto_Python_Procesamiento_de_datos/main/Data/cdmx_transporte_raw.csv"

In [3]:
data_raw = pd.read_csv(url)
data_raw.head(2)

,id,vendor_id,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,dist_meters,wait_sec
0,1,México DF Taxi de Sitio,2016-09-16 07:14:12,2016-09-18 04:41:40,-99.097369,19.416874,-99.202729,19.430353,N,120449,12373,242
1,2,México DF Taxi Libre,2016-09-18 06:16:33,2016-09-18 10:11:43,-99.297148,19.322128,-99.289949,19.326538,N,14110,1700,461


In [5]:
data_raw.dtypes

id                      int64
vendor_id              object
pickup_datetime        object
dropoff_datetime       object
pickup_longitude      float64
pickup_latitude       float64
dropoff_longitude     float64
dropoff_latitude      float64
store_and_fwd_flag     object
trip_duration           int64
dist_meters             int64
wait_sec                int64
dtype: object

### Descripción de las Columnas
* **id**: Un identificador único para cada viaje.
* **vendor_id**: El tipo de transporte en que se realizó el viaje.
* **pickup_datetime**: Fecha y Hora en que el viaje inició.
* **dropoff_datetime**: Fecha y Hora en que el viaje finalizó.
* **pickup_longitude**: Longitud en la que el viaje inició.
* **pickup_latitude**: Latitud en la que el viaje inició.
* **dropoff_longitude**: Longitud en la que el viaje finalizó.
* **dropoff_latitude**: Latitud en la que el viaje finalizó.
* **store_and_fwd_flag**: Indica si la información del viaje se almacenó en una memoria y después se envio al servidor por que no tenía conexión a internet o no. Todas las entradas de datos estan en 'N'.
* **trip_duration**: Duración del viaje en segundos.
* **dist_meters**: Distancia recorrida en el viaje en metros.
* **wait_sec**: Tiempo en segundos en el que el automovil estuvo completamente detenido durante el viaje. (Se usará como medición del tráfico.)